In [7]:
import sys
project_path = "/home/diego/Git/thesis-tabtrans"
sys.path.append(project_path) #import folders from the project_path

import os
import numpy as np
import torch
import torch.nn as nn
from utils import training, callback, evaluating, attention, data, plots 
from utils import fast_model as fm
from sklearn import datasets, model_selection
import skorch
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import openml
from sklearn import datasets, model_selection
from skorch.callbacks import Checkpoint, EarlyStopping, LoadInitState, EpochScoring, Checkpoint, TrainEndCheckpoint
import csv

In [8]:
task_id = 233093
layers = [4]
heads = [4]
embed_dim = 128 #The embedding size is set one by one to avoid the out of memory error
batch_size = 64
epochs = 10

In [18]:
128/4

32.0

In [9]:
    #parameters for the model
    ff_pw_size = 30  #this value because of the paper 
    attn_dropout = 0.3 #paper
    ff_dropout = 0.1 #paper value
    aggregator = "cls"
    aggregator_parameters = None
    decoder_hidden_units = [128,64] #paper value
    decoder_activation_fn = nn.ReLU()
    need_weights = False
    numerical_passthrough = False

In [10]:
#get the dataset_name
dataset_name = data.get_dataset_name(task_id)

In [11]:
X_train, X_test, y_train, y_test, n_instances, n_labels, n_numerical, n_categories = data.import_data(task_id)

INFO:openml.datasets.dataset:pickle write mfeat-factors


In [12]:
print(X_train.shape)
print(n_numerical)

(1600, 216)
216


In [13]:
train_indices, val_indices = model_selection.train_test_split(np.arange(X_train.shape[0]), test_size=0.333) #1/9 of train is equal to 10% of total


In [14]:
for n_layers in layers:
        for n_heads in heads:
                
            #module
            module = training.build_module(
                n_categories, # List of number of categories
                n_numerical, # Number of numerical features
                heads, # Number of heads per layer
                ff_pw_size, # Size of the MLP inside each transformer encoder layer
                n_layers, # Number of transformer encoder layers    
                n_labels, # Number of output neurons
                embed_dim,
                attn_dropout, 
                ff_dropout, 
                aggregator, # The aggregator for output vectors before decoder
                rnn_aggregator_parameters=aggregator_parameters,
                decoder_hidden_units=decoder_hidden_units,
                decoder_activation_fn=decoder_activation_fn,
                need_weights=need_weights,
                numerical_passthrough=numerical_passthrough
            )

/home/diego/anaconda3/envs/tabtrans/lib/python3.11/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer was not TransformerEncoderLayer
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [15]:
#MODEL
model = skorch.NeuralNetClassifier(
    module=module,
    criterion=torch.nn.CrossEntropyLoss,
    optimizer=torch.optim.AdamW,
    device="cpu", #"cuda" if torch.cuda.is_available() else "cpu",
    batch_size = batch_size,
    max_epochs = epochs,
    train_split=skorch.dataset.ValidSplit(((train_indices, val_indices),)),
    callbacks=[
        ("balanced_accuracy", skorch.callbacks.EpochScoring("balanced_accuracy", lower_is_better=False)),
        ("duration", skorch.callbacks.EpochTimer()),
        EpochScoring(scoring='accuracy', name='train_acc', on_train=True),
        Checkpoint(monitor='valid_acc_best',dirname = project_path, load_best = True),
        EarlyStopping(patience = 15)

    ],
    optimizer__lr=1e-4,
    optimizer__weight_decay=1e-4
)

In [16]:
# Define Checkpoint and TrainEndCheckpoint callbacks with custom directory
cp = Checkpoint()
train_end_cp = TrainEndCheckpoint()


In [17]:
#TRAINING
model = model.fit(X={
        "x_numerical": X_train[:, :n_numerical].astype(np.float32),
        "x_categorical": X_train[:, n_numerical:].astype(np.int32)
        }, 
        y=y_train.astype(np.int64)
        )

TypeError: unsupported operand type(s) for %: 'int' and 'list'